In [1]:
import time
import json
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.signal import butter, filtfilt
from scipy.interpolate import UnivariateSpline

In [2]:
def get_ts(date_str):
    ts = time.mktime(datetime.strptime(
        date_str, "%Y-%m-%d %H:%M:%S").timetuple())
    return int(ts)

In [3]:
def get_values_between_dates(matrix, sensor_idx, sensor_data, start_date, end_date):
    start_ts = get_ts(start_date)
    end_ts = get_ts(end_date)
    t = np.linspace(start_ts, end_ts, 100)
    result = {"t": t, "hps": []}
    for hp in range(10):
        df = sensor_data[matrix][sensor_idx][hp]
        df_interval = df[(df["Real time clock"] >= start_ts - 100)
                         & (df["Real time clock"] <= end_ts + 100)]

        rtc = df_interval["Real time clock"].values
        gas = df_interval["Resistance Gassensor"].values

        interpolation_func = UnivariateSpline(rtc, gas, s=0)

        y = interpolation_func(t)
        result["hps"].append(y)

    return result

In [4]:
def plot_sensor(mat, sensor, sensor_data):
    title = f"Matrix {mat}, Sensor {sensor}"
    fig = go.Figure()

    for i, data in enumerate(sensor_data["hps"]):
        fig.add_trace(go.Scatter(x=sensor_data["t"],
                                 y=data,
                                 mode="lines",
                                 showlegend=True,
                                 name=f"H.S. {i}"))

    fig.update_yaxes(title_text="Resistance (Ohms)",
                     type="log", tickformat=".0e")
    fig.update_xaxes(title_text="Datetime (Timestamp)", tickformat=".0e")
    fig.update_layout(height=400, width=700,
                      title_x=0.5,
                      font_family="Times New Roman",
                      title_font_family="Times New Roman",
                      title=dict(text=title, pad=dict(t=0, r=0, b=0, l=0)),
                      margin=dict(t=50, r=10, b=0, l=0))

    fig.update_yaxes(type="log")
    fig.show()

In [5]:
label_datetimes = [
    {
        "day": 0,
        "start": "2025-07-11 15:55:00",
        "end": "2025-07-11 16:15:00"
    },
    {
        "day": 1,
        "start": "2025-07-12 16:26:00",
        "end": "2025-07-12 16:46:00"
    },
    {
        "day": 2,
        "start": "2025-07-13 16:26:00",
        "end": "2025-07-13 16:46:00"
    },
    {
        "day": 3,
        "start": "2025-07-14 14:58:00",
        "end": "2025-07-14 15:18:00"
    }
]

In [9]:
DAY = 3
label = next((i for i in label_datetimes if i["day"] == DAY), None)

with open(f"data/day_{DAY}/raw.pkl", "rb") as f:
    raw_data = pickle.load(f)

interp_data = {"air": [], "sample": []}
for matrix in interp_data:
    for s_idx in range(8):
        s_data = get_values_between_dates(
            matrix,
            s_idx,
            raw_data,
            label["start"],
            label["end"])

        interp_data[matrix].append(s_data)

with open(f"data/day_{DAY}/interp.pkl", "wb") as f:
    pickle.dump(interp_data, f)